In [34]:
import pandas as pd
import re
import requests
from pathlib import Path
from urllib.parse import urlparse
import fitz  # PyMuPDF
import unicodedata
from rapidfuzz import fuzz
from openai import OpenAI
import os
pd.set_option("display.max_colwidth", None)
import json
import time
from IPython.display import clear_output


In [35]:
url = "https://diavgeia.gov.gr/luminapi/api/search"

params = {
    "fq": 'decisionType:["ΕΓΚΡΙΣΗ ΔΑΠΑΝΗΣ"]',
    "page": 0,
    "q": 'q:["πυροπροστ","αποψιλ", "δασοπροστ", "αντιπυρικ", "κατασβεσ"]',
    "sort": "recent",
    "size":100
}

In [36]:
url = "https://diavgeia.gov.gr/luminapi/api/search"

params = {
    "fq": 'organizationUid:"5002"',
    "page": 0,
    "q": 'q:["προσφυγικών πολυκατοικιών στη Λεωφόρο Αλεξάνδρας", "ΔΙΑΤΗΡΗΤΕΕΣ ΟΚΤΩ ΠΟΛΥΚΑΤΟΙΚΙΕΣ ΤΗΣ ΛΕΩΦΟΡΟΥ ΑΛΕΞΑΝΔΡΑΣ", "ΠΡΟΣΦΥΓΙΚΩΝ ΠΟΛΥΚΑΤΟΙΚΙΩΝ ΣΤΗ ΛΕΩΦΟΡΟ ΑΛΕΞΑΝΔΡΑΣ", "Προσφυγικών Πολυκατοικιών στη Λεωφ. Αλεξάνδρας", "ΠΡΟΣΦΥΓΙΚΩΝ ΠΟΛΥΚΑΤΟΙΚΙΩΝ ΣΤΗ Λ. ΑΛΕΞΑΝΔΡΑΣ"]',
    # "q": 'q:["προσφυγικων","πολυκατοικιων","αλεξανδρασ"]',
    "sort": "relative",
    "size":100
}

In [60]:
# Αναζήτηση για οικοδομικές άδειες από τεχνικό επιμελητήριο
url = "https://diavgeia.gov.gr/luminapi/api/search"

params = {
    "fq": [
        'thematicCategory:"περιβαλλον"',
        'organizationUid:"99201077"',
        'decisionTypeUid:"2.4.6.1"'
    ],
    "page": 0,
    "q": 'q:["προσφυγικών πολυκατοικιών στη Λεωφόρο Αλεξάνδρας", "ΔΙΑΤΗΡΗΤΕΕΣ ΟΚΤΩ ΠΟΛΥΚΑΤΟΙΚΙΕΣ ΤΗΣ ΛΕΩΦΟΡΟΥ ΑΛΕΞΑΝΔΡΑΣ", "ΠΡΟΣΦΥΓΙΚΩΝ ΠΟΛΥΚΑΤΟΙΚΙΩΝ ΣΤΗ ΛΕΩΦΟΡΟ ΑΛΕΞΑΝΔΡΑΣ", "Προσφυγικών Πολυκατοικιών στη Λεωφ. Αλεξάνδρας", "ΠΡΟΣΦΥΓΙΚΩΝ ΠΟΛΥΚΑΤΟΙΚΙΩΝ ΣΤΗ Λ. ΑΛΕΞΑΝΔΡΑΣ", "ΠΡΟΣΦΥΓΙΚΕΣ ΠΟΛΥΚΑΤΟΙΚΙΕΣ ΤΗΣ ΛΕΩΦ.ΑΛΕΞΑΝΔΡΑΣ"]',
    "sort": "relative",
    "size":100,
    f"page":{}
}

In [61]:
results = []
r = requests.get(url.format(0), params=params, headers={"Accept": "application/json"})
data = r.json()
total = data['info']['total']
pages = round(total/100)
results.extend(data['decisions'])
print(f"found {total} results")
print(len(results))
print(results)


found 1 results
1
[{'ada': '66ΚΞ46Ψ842-ΨΒ2', 'protocolNumber': '36956', 'issueDate': '20/05/2019 11:48:09', 'submissionTimestamp': '21/05/2019 10:50:39', 'publishTimestamp': '21/05/2019 10:50:39', 'status': 'PUBLISHED', 'nonRevokable': None, 'documentUrl': 'https://diavgeia.gov.gr/doc/66ΚΞ46Ψ842-ΨΒ2', 'subject': 'Οικοδομική άδεια Κατηγορίας 1 χωρίς προέγκριση: ΕΠΙΣΚΕΥΕΣ ΟΨΕΩΝ & ΕΣΩΤΕΡΙΚΕΣ ΔΙΑΡΡΥΘΜΙΣΕΙΣ ΣΗΜΕΙΑΚΩΝ ΕΠΕΜΒΑΣΕΩΝ ΣΤΙΣ ΟΚΤΩ (8) ΠΡΟΣΦΥΓΙΚΕΣ ΠΟΛΥΚΑΤΟΙΚΙΕΣ ΤΗΣ ΛΕΩΦ.ΑΛΕΞΑΝΔΡΑΣ', 'documentType': 'ΠΡΑΞΗ', 'versionComment': None, 'decisionType': {'uid': '2.4.6.1', 'label': 'ΠΡΑΞΕΙΣ ΧΩΡΟΤΑΞΙΚΟΥ - ΠΟΛΕΟΔΟΜΙΚΟΥ ΠΕΡΙΕΧΟΜΕΝΟΥ'}, 'thematicCategories': [{'uid': '52', 'label': 'ΠΕΡΙΒΑΛΛΟΝ'}], 'organization': {'uid': '99201077', 'label': 'ΤΕΧΝΙΚΟ ΕΠΙΜΕΛΗΤΗΡΙΟ ΕΛΛΑΔΑΣ', 'active': None, 'latinName': 'tee', 'category': None, 'inEffect': False}, 'cooperatingOrganizations': None, 'unitIds': ['77540']}]


In [ ]:
for i in range(1, pages+1):
    try:
        print(f"total results: {len(results)}")
        print(f"page {i}/{pages}")
        r = requests.get(url.format(i), params=params, headers={"Accept": "application/json"})
        data = r.json()
        results.extend(data['decisions'])
        
        clear_output(wait=True)
    except Exception as e:
        print(f"error: {e}")

page 9/477


KeyboardInterrupt: 

In [41]:
pd.DataFrame(results)

,ada,protocolNumber,issueDate,submissionTimestamp,publishTimestamp,status,nonRevokable,documentUrl,subject,documentType,versionComment,decisionType,thematicCategories,organization,cooperatingOrganizations,unitIds
0,ΡΙΕΕ46Ψ842-Υ54,636500,22/03/2023 15:04:51,22/03/2023 17:24:52,22/03/2023 17:24:52,PUBLISHED,None,https://diavgeia.gov.gr/doc/ΡΙΕΕ46Ψ842-Υ54,Έγκριση Εργασιών Δόμησης Μικρής Κλίμακας: ΑΝΑΚΑΙΝΙΣΗ ΟΨΕΩΝ ΣΥΜΦΩΝΑ ΜΕ ΤΟ ΠΡΟΓΡΑΜΜΑ ΠΡΟΣΟΨΗ ΣΕ ΠΟΛΥΚΑΤΟΙΚΙΑ ΕΠΙ ΤΗς ΛΕΩΦΟΡΟΥ ΑΛΕΞΑΝΔΡΑΣ,ΠΡΑΞΗ,None,"{'uid': '2.4.6.1', 'label': 'ΠΡΑΞΕΙΣ ΧΩΡΟΤΑΞΙΚΟΥ - ΠΟΛΕΟΔΟΜΙΚΟΥ ΠΕΡΙΕΧΟΜΕΝΟΥ'}","[{'uid': '52', 'label': 'ΠΕΡΙΒΑΛΛΟΝ'}]","{'uid': '99201077', 'label': 'ΤΕΧΝΙΚΟ ΕΠΙΜΕΛΗΤΗΡΙΟ ΕΛΛΑΔΑΣ', 'active': None, 'latinName': 'tee', 'category': None, 'inEffect': False}",None,[77540]
1,ΩΝ6Τ46Ψ842-ΦΝΩ,389444,09/04/2022 01:55:02,11/04/2022 13:55:06,11/04/2022 13:55:06,PUBLISHED,None,https://diavgeia.gov.gr/doc/ΩΝ6Τ46Ψ842-ΦΝΩ,"Αναθεώρηση Οικ. Αδείας ME προέγκριση ή βεβαίωση (ν.4759/2020): ΑΝΑΘΕΩΡΗΣΗ ΑΔΕΙΑΣ ""ΝΕΑ ΤΕΤΡΑΩΡΟΦΗ ΠΟΛΥΚΑΤΟΙΚΙΑ ΕΠΙ PILOTIS ΜΕ ΥΠΟΓΕΙΟ, ΓΚΑΡΑΖ, ΣΟΦΙΤΑ, ΣΤΕΓΗ & ΔΩΜΑ , ΓΙΑ ΔΙΟΡΘΩΣΗ ΠΑΡΑΤΗΡΗΣΕΩΝ ΕΛΕΓΚΤΗ ΔΟΜΗΣΗΣ",ΠΡΑΞΗ,None,"{'uid': '2.4.6.1', 'label': 'ΠΡΑΞΕΙΣ ΧΩΡΟΤΑΞΙΚΟΥ - ΠΟΛΕΟΔΟΜΙΚΟΥ ΠΕΡΙΕΧΟΜΕΝΟΥ'}","[{'uid': '52', 'label': 'ΠΕΡΙΒΑΛΛΟΝ'}]","{'uid': '99201077', 'label': 'ΤΕΧΝΙΚΟ ΕΠΙΜΕΛΗΤΗΡΙΟ ΕΛΛΑΔΑΣ', 'active': None, 'latinName': 'tee', 'category': None, 'inEffect': False}",None,None
2,6ΦΤ646Ψ842-0Χ2,417493,29/05/2022 20:17:23,30/05/2022 10:45:34,30/05/2022 10:45:34,PUBLISHED,None,https://diavgeia.gov.gr/doc/6ΦΤ646Ψ842-0Χ2,"Αναθεώρηση Οικ. Αδείας ME προέγκριση ή βεβαίωση (ν.4759/2020): ΑΝΑΘΕΩΡΗΣΗ ΤΗΣ ΥΠ.ΑΡ.372.855 ΕΝΗΜΕΡΩΣΗΣ ΤΗΣ ΟΙΚΟΔΟΜΙΚΗΣ ΑΔΕΙΑΣ 130.692/2020 ""ΝΕΑ TEΤΡΑΩΡΟΦΗ ΠΟΛΥΚΑΤΟΙΚΙΑ ΕΠΙ PILOTIS ΜΕ ΥΠΟΓΕΙΟ, ΓΚΑΡΑΖ, ΣΟΦΙΤΑ, ΣΤΕΓΗ & ΔΩΜΑ \nμε ΑΡΘΡΟ 25, ΠΑΡ. 2 (Προσαύξηση του Σ.Δ. κατα 10%, λόγω κατανάλωσης πρωτογενούς ενέργειας < 16%) και Αρθρο 15, παρ.8(Προσαύξηση ύψους κατά 1.00m, με μείωση κάλυψης κατά 5%)\n",ΠΡΑΞΗ,None,"{'uid': '2.4.6.1', 'label': 'ΠΡΑΞΕΙΣ ΧΩΡΟΤΑΞΙΚΟΥ - ΠΟΛΕΟΔΟΜΙΚΟΥ ΠΕΡΙΕΧΟΜΕΝΟΥ'}","[{'uid': '52', 'label': 'ΠΕΡΙΒΑΛΛΟΝ'}]","{'uid': '99201077', 'label': 'ΤΕΧΝΙΚΟ ΕΠΙΜΕΛΗΤΗΡΙΟ ΕΛΛΑΔΑΣ', 'active': None, 'latinName': 'tee', 'category': None, 'inEffect': False}",None,None
3,ΩΦ8646Ψ842-Ζ4Ω,302895,08/10/2021 09:20:03,08/10/2021 17:48:29,08/10/2021 17:48:29,PUBLISHED,None,https://diavgeia.gov.gr/doc/ΩΦ8646Ψ842-Ζ4Ω,Έγκριση Εργασιών Δόμησης Μικρής Κλίμακας: ΕΞΩΤΕΡΙΚΟΙ ΧΡΩΜΑΤΙΣΜΟΙ ΜΕ ΧΡΗΣΗ ΙΚΡΙΩΜΑΤΩΝ ΣΕ ΤΡΙΩΡΟΦΗ ΠΟΛΥΚΑΤΟΙΚΙΑ ΜΕ ΥΠΟΓΕΙΟ ΚΑΙ ΔΩΜΑ (ΚΤΙΡΙΟ Δ),ΠΡΑΞΗ,None,"{'uid': '2.4.6.1', 'label': 'ΠΡΑΞΕΙΣ ΧΩΡΟΤΑΞΙΚΟΥ - ΠΟΛΕΟΔΟΜΙΚΟΥ ΠΕΡΙΕΧΟΜΕΝΟΥ'}","[{'uid': '52', 'label': 'ΠΕΡΙΒΑΛΛΟΝ'}]","{'uid': '99201077', 'label': 'ΤΕΧΝΙΚΟ ΕΠΙΜΕΛΗΤΗΡΙΟ ΕΛΛΑΔΑΣ', 'active': None, 'latinName': 'tee', 'category': None, 'inEffect': False}",None,None
4,ΩΥΣ146Ψ842-Φ8Ψ,318352,08/11/2021 19:14:04,09/11/2021 15:49:42,09/11/2021 15:49:42,PUBLISHED,None,https://diavgeia.gov.gr/doc/ΩΥΣ146Ψ842-Φ8Ψ,Ενημέρωση Οικοδομικής Αδείας (ν.4759/2020): ΝΕΑ 7ΟΡΟΦΗ ΠΟΛΥΚΑΤΟΙΚΙΑ ΜΕ PILOTIS & ΥΠΟΓΕΙΟ \n,ΠΡΑΞΗ,None,"{'uid': '2.4.6.1', 'label': 'ΠΡΑΞΕΙΣ ΧΩΡΟΤΑΞΙΚΟΥ - ΠΟΛΕΟΔΟΜΙΚΟΥ ΠΕΡΙΕΧΟΜΕΝΟΥ'}","[{'uid': '52', 'label': 'ΠΕΡΙΒΑΛΛΟΝ'}]","{'uid': '99201077', 'label': 'ΤΕΧΝΙΚΟ ΕΠΙΜΕΛΗΤΗΡΙΟ ΕΛΛΑΔΑΣ', 'active': None, 'latinName': 'tee', 'category': None, 'inEffect': False}",None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,ΨΕΦ046Ψ842-ΤΥΒ,912865,09/01/2024 12:36:00,09/01/2024 14:33:00,09/01/2024 14:33:00,PUBLISHED,None,https://diavgeia.gov.gr/doc/ΨΕΦ046Ψ842-ΤΥΒ,Έγκριση Εργασιών Δόμησης Μικρής Κλίμακας: ΕΠΙΣΚΕΥΗ ΟΨΕΩΝ ΣΕ ΠΕΝΤΑΟΡΟΦΗ ΠΟΛΥΚΑΤΟΙΚΙΑ ΧΩΡΙΣ ΠΑΡΕΜΒΑΣΕΙΣ ΚΑΙ ΑΛΛΑΓΕΣ ΑΥΤΩΝ.,ΠΡΑΞΗ,None,"{'uid': '2.4.6.1', 'label': 'ΠΡΑΞΕΙΣ ΧΩΡΟΤΑΞΙΚΟΥ - ΠΟΛΕΟΔΟΜΙΚΟΥ ΠΕΡΙΕΧΟΜΕΝΟΥ'}","[{'uid': '52', 'label': 'ΠΕΡΙΒΑΛΛΟΝ'}]","{'uid': '99201077', 'label': 'ΤΕΧΝΙΚΟ ΕΠΙΜΕΛΗΤΗΡΙΟ ΕΛΛΑΔΑΣ', 'active': None, 'latinName': 'tee', 'category': None, 'inEffect': False}",None,None
896,Ρ6Χ946Ψ842-ΗΦΚ,980388,11/03/2024 13:12:56,11/03/2024 16:51:53,11/03/2024 16:51:53,PU

In [26]:
print(data.keys())
print(data['errors'])

dict_keys(['decisions', 'facets', 'highlighting', 'info'])


KeyError: 'errors'

In [27]:
data['info']

{'query': None,
 'page': 0,
 'size': 100,
 'actualSize': 100,
 'total': 47739,
 'order': None}

In [23]:
pd.DataFrame(data['decisions']).head(2)

,ada,protocolNumber,issueDate,submissionTimestamp,publishTimestamp,status,nonRevokable,documentUrl,subject,documentType,versionComment,decisionType,thematicCategories,organization,cooperatingOrganizations,unitIds
0,ΡΙΕΕ46Ψ842-Υ54,636500,22/03/2023 15:04:51,22/03/2023 17:24:52,22/03/2023 17:24:52,PUBLISHED,None,https://diavgeia.gov.gr/doc/ΡΙΕΕ46Ψ842-Υ54,Έγκριση Εργασιών Δόμησης Μικρής Κλίμακας: ΑΝΑΚΑΙΝΙΣΗ ΟΨΕΩΝ ΣΥΜΦΩΝΑ ΜΕ ΤΟ ΠΡΟΓΡΑΜΜΑ ΠΡΟΣΟΨΗ ΣΕ ΠΟΛΥΚΑΤΟΙΚΙΑ ΕΠΙ ΤΗς ΛΕΩΦΟΡΟΥ ΑΛΕΞΑΝΔΡΑΣ,ΠΡΑΞΗ,None,"{'uid': '2.4.6.1', 'label': 'ΠΡΑΞΕΙΣ ΧΩΡΟΤΑΞΙΚΟΥ - ΠΟΛΕΟΔΟΜΙΚΟΥ ΠΕΡΙΕΧΟΜΕΝΟΥ'}","[{'uid': '52', 'label': 'ΠΕΡΙΒΑΛΛΟΝ'}]","{'uid': '99201077', 'label': 'ΤΕΧΝΙΚΟ ΕΠΙΜΕΛΗΤΗΡΙΟ ΕΛΛΑΔΑΣ', 'active': None, 'latinName': 'tee', 'category': None, 'inEffect': False}",None,[77540]
1,ΩΝ6Τ46Ψ842-ΦΝΩ,389444,09/04/2022 01:55:02,11/04/2022 13:55:06,11/04/2022 13:55:06,PUBLISHED,None,https://diavgeia.gov.gr/doc/ΩΝ6Τ46Ψ842-ΦΝΩ,"Αναθεώρηση Οικ. Αδείας ME προέγκριση ή βεβαίωση (ν.4759/2020): ΑΝΑΘΕΩΡΗΣΗ ΑΔΕΙΑΣ ""ΝΕΑ ΤΕΤΡΑΩΡΟΦΗ ΠΟΛΥΚΑΤΟΙΚΙΑ ΕΠΙ PILOTIS ΜΕ ΥΠΟΓΕΙΟ, ΓΚΑΡΑΖ, ΣΟΦΙΤΑ, ΣΤΕΓΗ & ΔΩΜΑ , ΓΙΑ ΔΙΟΡΘΩΣΗ ΠΑΡΑΤΗΡΗΣΕΩΝ ΕΛΕΓΚΤΗ ΔΟΜΗΣΗΣ",ΠΡΑΞΗ,None,"{'uid': '2.4.6.1', 'label': 'ΠΡΑΞΕΙΣ ΧΩΡΟΤΑΞΙΚΟΥ - ΠΟΛΕΟΔΟΜΙΚΟΥ ΠΕΡΙΕΧΟΜΕΝΟΥ'}","[{'uid': '52', 'label': 'ΠΕΡΙΒΑΛΛΟΝ'}]","{'uid': '99201077', 'label': 'ΤΕΧΝΙΚΟ ΕΠΙΜΕΛΗΤΗΡΙΟ ΕΛΛΑΔΑΣ', 'active': None, 'latinName': 'tee', 'category': None, 'inEffect': False}",None,None


In [21]:
results=[]
for i in data['decisions']:
    result = {}
    result['label'] = i['organization']['label']
    result['subject'] = i['subject']
    result['ada'] = i['ada']
    result['issueDate'] = i['issueDate']
    result['documentUrl'] = i['documentUrl']
    result['documentType'] = i['documentType']
    result['decisionType'] = i["decisionType"]["label"]
    results.append(result)

In [22]:
df1 = pd.DataFrame(results)

In [23]:
df1.head(1)

,label,subject,ada,issueDate,documentUrl,documentType,decisionType
0,ΠΕΡΙΦΕΡΕΙΑ ΑΤΤΙΚΗΣ,"Κ.Τ. ΚΑΕ 97690113701 ΔΕΣΜΕΥΣΗ ΠΟΛΥΕΤΟΥΣ ΥΠΟΧΡΕΩΣΗΣ\n""ΥΠΗΡΕΣΙΕΣ ΣΥΜΒΟΥΛΟΥ ΓΙΑ ΤΗΝ ΩΡΙΜΑΝΣΗ ΤΩΝ ΕΡΓΩΝ ΑΠΟΚΑΤΑΣΤΑΣΗΣ ΤΩΝ ΠΡΟΣΦΥΓΙΚΩΝ ΠΟΛΥΚΑΤΟΙΚΙΩΝ ΣΤΗ ΛΕΩΦΟΡΟ ΑΛΕΞΑΝΔΡΑΣ",7Θ3Ν7Λ7-Υ4Ν,11/10/2017 03:00:00,https://diavgeia.gov.gr/doc/7Θ3Ν7Λ7-Υ4Ν,ΠΡΑΞΗ,ΑΝΑΛΗΨΗ ΥΠΟΧΡΕΩΣΗΣ


In [24]:
save_dir = Path("data/pdfs")
save_dir.mkdir(parents=True, exist_ok=True)

for i in results:
    filename = Path(urlparse(i["documentUrl"]).path).name or "downloaded.pdf"
    if not filename.lower().endswith(".pdf"):
        filename += ".pdf"

    save_path = save_dir / filename

    if save_path.exists():
        print(f"already exists, skipping: {save_path.name}")
        continue

    r = requests.get(i["documentUrl"], timeout=30)
    r.raise_for_status()

    with open(save_path, "wb") as f:
        f.write(r.content)

    print(f"saved to: {save_path.resolve()}")

already exists, skipping: 7Θ3Ν7Λ7-Υ4Ν.pdf
already exists, skipping: Ω0ΦΝ7Λ7-8ΔΔ.pdf
already exists, skipping: ΨΠ6Ρ7Λ7-579.pdf
already exists, skipping: 6Ε537Λ7-Π2Σ.pdf
already exists, skipping: 9Ξ7Ω7Λ7-Κ92.pdf
already exists, skipping: Ω70Ε7Λ7-00Λ.pdf
already exists, skipping: ΩΣ5Η7Λ7-ΖΕΠ.pdf
already exists, skipping: ΨΤΣΙ7Λ7-ΧΔ6.pdf
already exists, skipping: 61ΛΩ7Λ7-4ΜΥ.pdf
already exists, skipping: ΩΣΕ47Λ7-ΨΨ4.pdf
already exists, skipping: 6Ω8Ξ7Λ7-80Θ.pdf
already exists, skipping: ΩΘΩ77Λ7-Ο0Μ.pdf
already exists, skipping: 9ΙΘΘ7Λ7-ΟΩΖ.pdf
already exists, skipping: Ψ24Π7Λ7-ΑΕ9.pdf
already exists, skipping: ΩΞ637Λ7-ΔΣΡ.pdf
already exists, skipping: Ω0ΨΛ7Λ7-Ζ7Ρ.pdf
already exists, skipping: 6ΙΘΩ7Λ7-ΨΙ3.pdf
already exists, skipping: ΨΣΜΠ7Λ7-Γ6Ω.pdf
already exists, skipping: 954Γ7Λ7-ΔΣ6.pdf
already exists, skipping: 9Ι1Ρ7Λ7-Α1Μ.pdf
already exists, skipping: Ω8ΟΥ7Λ7-Κ5Κ.pdf
already exists, skipping: ΩΖ177Λ7-Δ1Γ.pdf
already exists, skipping: ΡΨΩΝ7Λ7-0ΔΓ.pdf
already exists, skipping: 6Θ4Β7Λ7-

In [25]:
import os
print(bool(os.getenv("OPENAI_API_KEY")))


True


In [29]:
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Set OPENAI_API_KEY in environment (or .env) before running this cell.")

client = OpenAI(api_key=api_key)
pdf_dir = Path("data/pdfs")

terms = [
    "προσφυγικών πολυκατοικιών στη Λεωφόρο Αλεξάνδρας",
    "ΔΙΑΤΗΡΗΤΕΕΣ ΟΚΤΩ ΠΟΛΥΚΑΤΟΙΚΙΕΣ ΤΗΣ ΛΕΩΦΟΡΟΥ ΑΛΕΞΑΝΔΡΑΣ",
    "ΠΡΟΣΦΥΓΙΚΩΝ ΠΟΛΥΚΑΤΟΙΚΙΩΝ ΣΤΗ ΛΕΩΦΟΡΟ ΑΛΕΞΑΝΔΡΑΣ",
    "Προσφυγικών Πολυκατοικιών στη Λεωφ. Αλεξάνδρας",
    "ΠΡΟΣΦΥΓΙΚΩΝ ΠΟΛΥΚΑΤΟΙΚΙΩΝ ΣΤΗ Λ. ΑΛΕΞΑΝΔΡΑΣ",
]

def pdf_text(path: Path, max_chars: int = 12000, max_chars_per_page: int = 10000) -> str:
    out = []
    used = 0
    with fitz.open(path) as doc:
        for i, p in enumerate(doc, start=1):
            t = re.sub(r"\s+", " ", p.get_text("text")).strip()
            if not t:
                continue
            if len(t) > max_chars_per_page:
                t = t[:max_chars_per_page] + " ..."
            page_block = f"[PAGE {i}] {t}"

            remaining = max_chars - used
            if remaining <= 0:
                break
            if len(page_block) > remaining:
                out.append(page_block[:remaining] + " ...")
                break

            out.append(page_block)
            used += len(page_block) + 1

    return "\n".join(out)

rows = []

for pdf in sorted(pdf_dir.glob("*.pdf")):
    print("Processing:", pdf.name)
    text = pdf_text(pdf, max_chars=120000, max_chars_per_page=10000)

    prompt = f"""
Όροι:
{terms}

Κείμενο:
{text}

Βήματα:
1. Διάβασε και κατάλαβε καλά τι περιέχει το έγγραφο.
2. Θα επιστρέψεις ένα json με τα εξής keys: relevance, description, table, companies, pages, extras
3. Αν το έγγραφο περιέχει πληροφορίες για τα προσφυγικά της αλεξάνδρας, επέστρεψε relevance: YES. Σε κάθε άλλη περίπτωση relevance: NO
4. description: [περιγραφή έως 100 λέξεις για το τί αναφέρεται για τις προσφυγικές πολυκατοικίες της λ. αλεξάνδρας]
5. Εάν το έγγραφο περιέχει οικονομικά στοιχεία για τα προσφυγικά της λ. αλεξάνδρας τότε, table : [οι γραμμές με τα οικονομικά στοιχεία που αναφέρονται για τα προσφυγικά της λεωφόρου αλεξάνδρας].
6. companies: [Οι εταιρείες που αναφέρονται στο έγγραφο και έχουν σχέση με τα προσφυγικά της λεωφόρου αλεξάνδρας (NER)]
7. pages: [οι σελίδες στις οποίες αναφέρονται οι προσφυγικές πολυκατοικίες της λεωφόρου αλεξάνδρας]
8. εάν το έγγραφο περιέχουν πρακτικά, περιέγραψε τι λέγεται για τα προσφυγικά της λεωφόρου αλεξάνδρας, extras:[περιγραφή των πρακτικών]
9. διάβασε το json ξανά και αφαίρεσε σχόλια σου που πιθανώς έχεις προσθέσει. Το json πρέπει να μην περιέχει σχόλιά σου διότι ενημερώνει αυτόματα ένα web app που διαβάζουν οι αναγνώστες.

"""

    try:
        resp = client.responses.create(
            model="gpt-5-mini",
            input=prompt
        )
    except Exception as e:
        msg = str(e)
        if "context_length_exceeded" in msg or "context window" in msg:
            print("Retrying with smaller context:", pdf.name)
            text = pdf_text(pdf, max_chars=120000, max_chars_per_page=10000)
            prompt = prompt.replace(
                "Αποσπάσματα κειμένου (ενδέχεται περικομμένα για όριο context):\n" + prompt.split("Αποσπάσματα κειμένου (ενδέχεται περικομμένα για όριο context):\n", 1)[1].split("\n\nΒήματα:", 1)[0],
                f"Αποσπάσματα κειμένου (ενδέχεται περικομμένα για όριο context):\n{text}"
            )
            resp = client.responses.create(
                model="gpt-5-mini",
                input=prompt
            )
        else:
            raise

    print(resp.output_text)
    rows.append({"file": pdf.stem, "summary": resp.output_text})

df = pd.DataFrame(rows)
display(df)
df.to_csv("data/openai_simple_summaries.csv", index=False, encoding="utf-8-sig")


Processing: 607Γ7Λ7-ΥΡΙ.pdf
{
  "relevance": "YES",
  "description": "Το έγγραφο καταγράφει ως ενταγμένο έργο την «Αποκατάσταση προσφυγικών πολυκατοικιών στη Λ. Αλεξάνδρας και ανάπλαση περιβάλλοντα χώρου», με προτεινόμενο προϋπολογισμό 46.000.000€, νομική δέσμευση 46.000.000€ και εγκεκριμένες πιστώσεις 2019 50.000€ (προτεινόμενες πιστώσεις 1.000€).",
  "table": [
    {
      "ΤΙΤΛΟΣ_ΕΡΓΟΥ": "ΑΠΟΚΑΤΑΣΤΑΣΗ ΠΡΟΣΦΥΓΙΚΩΝ ΠΟΛΥΚΑΤΟΙΚΙΩΝ ΣΤΗ Λ. ΑΛΕΞΑΝΔΡΑΣ ΚΑΙ ΑΝΑΠΛΑΣΗ ΠΕΡΙΒΑΛΛΟΝΤΑ ΧΩΡΟΥ",
      "ΚΑΕ": "9729.01.024",
      "ΕΙΔΟΣ_ΕΡΓΟΥ": "ΑΣΤΙΚΕΣ ΑΝΑΠΛΑΣΕΙΣ",
      "ΦΟΡΕΑΣ_ΑΡΜΟΔΙΟΤΗΤΑΣ": "ΠΕΡΙΦΕΡΕΙΑ ΑΤΤΙΚΗΣ",
      "ΦΟΡΕΑΣ_ΥΛΟΠΟΙΗΣΗΣ": "ΠΕΡΙΦΕΡΕΙΑ ΑΤΤΙΚΗΣ",
      "ΠΡΟΫΠΟΛΟΓΙΣΜΟΣ_ΠΡΟ_ΔΗΜΟΠΡΑΤΗΣΗΣ_(με_ΦΠΑ)": "46.000.000,00",
      "ΕΓΚΕΚΡΙΜΕΝΟΣ_ΠΡΟΫΠΟΛΟΓΙΣΜΟΣ_(ΝΟΜΙΚΗ_ΔΕΣΜΕΥΣΗ)": "46.000.000,00",
      "ΠΡΟΤΕΙΝΟΜΕΝΟΣ_ΠΡΟΫΠΟΛΟΓΙΣΜΟΣ_ΕΡΓΟΥ_(ΝΟΜΙΚΗ_ΔΕΣΜΕΥΣΗ)": "46.000.000,00",
      "ΣΥΝΟΛΟ_ΠΛΗΡΩΜΩΝ_ΕΩΣ_31-12-2018": "0,00",
      "ΥΠΟΛΟΙΠΟ_ΠΡΟΫΠΟΛΟΓΙΣΜΟΥ_(Ν/Δ)_01-01-2019": "46.000.000,00",
      "ΕΓΚ

,file,summary
0,607Γ7Λ7-ΥΡΙ,"{\n ""relevance"": ""YES"",\n ""description"": ""Το έγγραφο καταγράφει ως ενταγμένο έργο την «Αποκατάσταση προσφυγικών πολυκατοικιών στη Λ. Αλεξάνδρας και ανάπλαση περιβάλλοντα χώρου», με προτεινόμενο προϋπολογισμό 46.000.000€, νομική δέσμευση 46.000.000€ και εγκεκριμένες πιστώσεις 2019 50.000€ (προτεινόμενες πιστώσεις 1.000€)."",\n ""table"": [\n {\n ""ΤΙΤΛΟΣ_ΕΡΓΟΥ"": ""ΑΠΟΚΑΤΑΣΤΑΣΗ ΠΡΟΣΦΥΓΙΚΩΝ ΠΟΛΥΚΑΤΟΙΚΙΩΝ ΣΤΗ Λ. ΑΛΕΞΑΝΔΡΑΣ ΚΑΙ ΑΝΑΠΛΑΣΗ ΠΕΡΙΒΑΛΛΟΝΤΑ ΧΩΡΟΥ"",\n ""ΚΑΕ"": ""9729.01.024"",\n ""ΕΙΔΟΣ_ΕΡΓΟΥ"": ""ΑΣΤΙΚΕΣ ΑΝΑΠΛΑΣΕΙΣ"",\n ""ΦΟΡΕΑΣ_ΑΡΜΟΔΙΟΤΗΤΑΣ"": ""ΠΕΡΙΦΕΡΕΙΑ ΑΤΤΙΚΗΣ"",\n ""ΦΟΡΕΑΣ_ΥΛΟΠΟΙΗΣΗΣ"": ""ΠΕΡΙΦΕΡΕΙΑ ΑΤΤΙΚΗΣ"",\n ""ΠΡΟΫΠΟΛΟΓΙΣΜΟΣ_ΠΡΟ_ΔΗΜΟΠΡΑΤΗΣΗΣ_(με_ΦΠΑ)"": ""46.000.000,00"",\n ""ΕΓΚΕΚΡΙΜΕΝΟΣ_ΠΡΟΫΠΟΛΟΓΙΣΜΟΣ_(ΝΟΜΙΚΗ_ΔΕΣΜΕΥΣΗ)"": ""46.000.000,00"",\n ""ΠΡΟΤΕΙΝΟΜΕΝΟΣ_ΠΡΟΫΠΟΛΟΓΙΣΜΟΣ_ΕΡΓΟΥ_(ΝΟΜΙΚΗ_ΔΕΣΜΕΥΣΗ)"": ""46.000.000,00"",\n ""ΣΥΝΟΛΟ_ΠΛΗΡΩΜΩΝ_ΕΩΣ_31-12-2018"": ""0,00"",\n ""ΥΠΟΛΟΙΠΟ_ΠΡΟΫΠΟΛΟΓΙΣΜΟΥ_(Ν/Δ)_01-01-2019"": ""46.000.000,00"",\n ""ΕΓΚΕΚΡΙΜΕΝΕΣ_ΠΙΣΤΩΣΕΙΣ_2019"": ""50.000,00"",\n ""ΠΡΟΤΕΙΝΟΜΕΝΕΣ_ΠΙΣΤΩΣΕΙΣ_2019"": ""1.000,00"",\n ""ΜΕΤΑΒΟΛΗ_ΠΙΣΤΩΣΕΩΝ"": ""-49.000,00"",\n ""ΠΛΗΘΟΣ_ΕΡΓΩΝ"": ""1"",\n ""page"": 16,\n ""raw_row"": ""ΑΠΟΚΑΤΑΣΤΑΣΗ ΠΡΟΣΦΥΓΙΚΩΝ ΠΟΛΥΚΑΤΟΙΚΙΩΝ ΣΤΗ Λ. ΑΛΕΞΑΝΔΡΑΣ ΚΑΙ ΑΝΑΠΛΑΣΗ ΠΕΡΙΒΑΛΛΟΝΤΑ ΧΩΡΟΥ. 9729.01.024 ΑΣΤΙΚΕΣ ΑΝΑΠΛΑΣΕΙΣ ΠΕΡΙΦΕΡΕΙΑ ΑΤΤΙΚΗΣ ΠΕΡΙΦΕΡΕΙΑ ΑΤΤΙΚΗΣ 46.000.000,00 46.000.000,00 46.000.000,00 0,00 46.000.000,00 50.000,00 1.000,00 -49.000,00 1""\n }\n ],\n ""companies"": [],\n ""pages"": [\n 16\n ],\n ""extras"": ""Το έγγραφο είναι πρακτικά (απόφαση / συνεδρίαση του Περιφερειακού Συμβουλίου Αττικής για τη 2η τροποποίηση του Προγράμματος Εκτελεστέων Έργων 2019). Το έργο των προσφυγικών της Λ. Αλεξάνδρας περιλαμβάνεται στους πίνακες (Πίνακας 4) με οικονομικά στοιχεία και προτεινόμενες πιστώσεις. Στα διαθέσιμα αποσπάσματα δεν καταγράφεται ειδική συζήτηση ή απόφαση επί του συγκεκριμένου έργου πέραν της καταγραφής του στον πίνακα τροποποιήσεων/προϋπολογισμών.""\n}"
1,61ΛΩ7Λ7-4ΜΥ,"{\n ""relevance"": ""YES"",\n ""description"": ""Έγγραφο της Περιφέρειας Αττικής για παράταση σύμβασης παροχής υπηρεσιών συμβούλου που αφορά την ωρίμανση των έργων αποκατάστασης των Προσφυγικών Πολυκατοικιών στη Λεωφ. Αλεξάνδρας. Αναφέρεται η σύμβαση με την ανάδοχο κοινοπραξία, η πρόοδος των μελετών (επισκευών όψεων κ.ά.) και η πρόταση/έγκριση παράτασης της συνολικής προθεσμίας κατά 6 μήνες έως 10-5-2021. Επισημαίνεται δυσκολία πρόσβασης λόγω καταλήψεων και εκκρεμότητες για υπόγειο χώρο στάθμευσης."",\n ""table"": [\n ""Συμβατική αμοιβή: 199.099,50€ πλέον ΦΠΑ"",\n ""Αρχική διάρκεια σύμβασης: 12 μήνες (υπέγραψε 10-7-2019 έως 10-7-2020)"",\n ""Προηγούμενη παράταση: +4 μήνες (έως 10-11-2020) – Απόφαση 1530/2020"",\n ""Νέα εγκεκριμένη παράταση: +6 μήνες (έως 10-5-2021)"",\n ""Ποσοστό ολοκλήρωσης φυσικού αντικειμένου: ~90%"",\n ""Ποσοστό ολοκλήρωσης οικονομικού αντικειμένου: 64%""\n ],\n ""companies"": [\n ""ΑΝΑΠΛΑΣΗ Α.Ε."",\n ""MELCON ENGINEERING Ε.Ε."",\n ""ΒΑΦΕΙΑΔΗ ΕΜΜΑΝΟΥΕΛΑ""\n ],\n ""pages"": [\n 1,\n 2,\n 3,\n 4\n ],\n ""extras"": ""Τα πρακτικά και σημειώσεις αναφέρουν ότι ο ανάδοχος έχει υποβάλει 15 μηνιαίες τεχνικές εκθέσεις, τα πρώτα 14 παραδοτέα αξιολογήθηκαν σε πέντε πρακτικά και πιστοποιήθηκε η παροχή υπηρεσιών. Η μελέτη εφαρμογής για τις όψεις έχει εκπονηθεί όσο ήταν δυνατό λόγω κατάληψης κτιρίων (πρόσβαση σε ένα μόνο κτίριο) και δεν έγινε οριστική παραλαβή. Για τον υπόγειο χώρο στάθμευσης υπάρχουν ολοκληρωμένες προμελέτες και γεωτεχνική έρευνα, αλλά εκκρεμεί τροποποίηση ρυμοτομικού, τρόπος/χρόνος δημοπράτησης και πιθανή προγραμματική σύμβαση με το Δήμο Αθηναίων — εκκρεμότητες εκτός συμβατικού αντικειμένου. Η Οικονομική Επιτροπή ενέκρινε κατά πλειοψηφία την παράταση· ένας απέχει και δύο αναπληρωματικοί καταψήφισαν με αιτιολογία έλλειψης γραπτής εισήγησης και επαρκών εξηγήσεων.""\n}"
2,61ΡΣ7Λ7-ΝΚΒ,"{\n ""relevance"": ""YES"",\n ""description"": ""Απόφαση συγκρότησης επιτροπής για την παραλαβή, έλεγχο και εισήγηση έγκρισης της μελέτης «Επισκευές όψεων, εσωτερικές 

In [30]:
df.head(1)

,file,summary
0,607Γ7Λ7-ΥΡΙ,"{\n ""relevance"": ""YES"",\n ""description"": ""Το έγγραφο καταγράφει ως ενταγμένο έργο την «Αποκατάσταση προσφυγικών πολυκατοικιών στη Λ. Αλεξάνδρας και ανάπλαση περιβάλλοντα χώρου», με προτεινόμενο προϋπολογισμό 46.000.000€, νομική δέσμευση 46.000.000€ και εγκεκριμένες πιστώσεις 2019 50.000€ (προτεινόμενες πιστώσεις 1.000€)."",\n ""table"": [\n {\n ""ΤΙΤΛΟΣ_ΕΡΓΟΥ"": ""ΑΠΟΚΑΤΑΣΤΑΣΗ ΠΡΟΣΦΥΓΙΚΩΝ ΠΟΛΥΚΑΤΟΙΚΙΩΝ ΣΤΗ Λ. ΑΛΕΞΑΝΔΡΑΣ ΚΑΙ ΑΝΑΠΛΑΣΗ ΠΕΡΙΒΑΛΛΟΝΤΑ ΧΩΡΟΥ"",\n ""ΚΑΕ"": ""9729.01.024"",\n ""ΕΙΔΟΣ_ΕΡΓΟΥ"": ""ΑΣΤΙΚΕΣ ΑΝΑΠΛΑΣΕΙΣ"",\n ""ΦΟΡΕΑΣ_ΑΡΜΟΔΙΟΤΗΤΑΣ"": ""ΠΕΡΙΦΕΡΕΙΑ ΑΤΤΙΚΗΣ"",\n ""ΦΟΡΕΑΣ_ΥΛΟΠΟΙΗΣΗΣ"": ""ΠΕΡΙΦΕΡΕΙΑ ΑΤΤΙΚΗΣ"",\n ""ΠΡΟΫΠΟΛΟΓΙΣΜΟΣ_ΠΡΟ_ΔΗΜΟΠΡΑΤΗΣΗΣ_(με_ΦΠΑ)"": ""46.000.000,00"",\n ""ΕΓΚΕΚΡΙΜΕΝΟΣ_ΠΡΟΫΠΟΛΟΓΙΣΜΟΣ_(ΝΟΜΙΚΗ_ΔΕΣΜΕΥΣΗ)"": ""46.000.000,00"",\n ""ΠΡΟΤΕΙΝΟΜΕΝΟΣ_ΠΡΟΫΠΟΛΟΓΙΣΜΟΣ_ΕΡΓΟΥ_(ΝΟΜΙΚΗ_ΔΕΣΜΕΥΣΗ)"": ""46.000.000,00"",\n ""ΣΥΝΟΛΟ_ΠΛΗΡΩΜΩΝ_ΕΩΣ_31-12-2018"": ""0,00"",\n ""ΥΠΟΛΟΙΠΟ_ΠΡΟΫΠΟΛΟΓΙΣΜΟΥ_(Ν/Δ)_01-01-2019"": ""46.000.000,00"",\n ""ΕΓΚΕΚΡΙΜΕΝΕΣ_ΠΙΣΤΩΣΕΙΣ_2019"": ""50.000,00"",\n ""ΠΡΟΤΕΙΝΟΜΕΝΕΣ_ΠΙΣΤΩΣΕΙΣ_2019"": ""1.000,00"",\n ""ΜΕΤΑΒΟΛΗ_ΠΙΣΤΩΣΕΩΝ"": ""-49.000,00"",\n ""ΠΛΗΘΟΣ_ΕΡΓΩΝ"": ""1"",\n ""page"": 16,\n ""raw_row"": ""ΑΠΟΚΑΤΑΣΤΑΣΗ ΠΡΟΣΦΥΓΙΚΩΝ ΠΟΛΥΚΑΤΟΙΚΙΩΝ ΣΤΗ Λ. ΑΛΕΞΑΝΔΡΑΣ ΚΑΙ ΑΝΑΠΛΑΣΗ ΠΕΡΙΒΑΛΛΟΝΤΑ ΧΩΡΟΥ. 9729.01.024 ΑΣΤΙΚΕΣ ΑΝΑΠΛΑΣΕΙΣ ΠΕΡΙΦΕΡΕΙΑ ΑΤΤΙΚΗΣ ΠΕΡΙΦΕΡΕΙΑ ΑΤΤΙΚΗΣ 46.000.000,00 46.000.000,00 46.000.000,00 0,00 46.000.000,00 50.000,00 1.000,00 -49.000,00 1""\n }\n ],\n ""companies"": [],\n ""pages"": [\n 16\n ],\n ""extras"": ""Το έγγραφο είναι πρακτικά (απόφαση / συνεδρίαση του Περιφερειακού Συμβουλίου Αττικής για τη 2η τροποποίηση του Προγράμματος Εκτελεστέων Έργων 2019). Το έργο των προσφυγικών της Λ. Αλεξάνδρας περιλαμβάνεται στους πίνακες (Πίνακας 4) με οικονομικά στοιχεία και προτεινόμενες πιστώσεις. Στα διαθέσιμα αποσπάσματα δεν καταγράφεται ειδική συζήτηση ή απόφαση επί του συγκεκριμένου έργου πέραν της καταγραφής του στον πίνακα τροποποιήσεων/προϋπολογισμών.""\n}"


In [31]:
df1.head(1)

,label,subject,ada,issueDate,documentUrl,documentType,decisionType
0,ΠΕΡΙΦΕΡΕΙΑ ΑΤΤΙΚΗΣ,"Κ.Τ. ΚΑΕ 97690113701 ΔΕΣΜΕΥΣΗ ΠΟΛΥΕΤΟΥΣ ΥΠΟΧΡΕΩΣΗΣ\n""ΥΠΗΡΕΣΙΕΣ ΣΥΜΒΟΥΛΟΥ ΓΙΑ ΤΗΝ ΩΡΙΜΑΝΣΗ ΤΩΝ ΕΡΓΩΝ ΑΠΟΚΑΤΑΣΤΑΣΗΣ ΤΩΝ ΠΡΟΣΦΥΓΙΚΩΝ ΠΟΛΥΚΑΤΟΙΚΙΩΝ ΣΤΗ ΛΕΩΦΟΡΟ ΑΛΕΞΑΝΔΡΑΣ",7Θ3Ν7Λ7-Υ4Ν,11/10/2017 03:00:00,https://diavgeia.gov.gr/doc/7Θ3Ν7Λ7-Υ4Ν,ΠΡΑΞΗ,ΑΝΑΛΗΨΗ ΥΠΟΧΡΕΩΣΗΣ


In [32]:
final = pd.merge(df1, df, how='inner', left_on="ada", right_on="file")

In [33]:
final.to_csv("data_alexandras.csv", index=False)

In [34]:
final.head(1)

,label,subject,ada,issueDate,documentUrl,documentType,decisionType,file,summary
0,ΠΕΡΙΦΕΡΕΙΑ ΑΤΤΙΚΗΣ,"Κ.Τ. ΚΑΕ 97690113701 ΔΕΣΜΕΥΣΗ ΠΟΛΥΕΤΟΥΣ ΥΠΟΧΡΕΩΣΗΣ\n""ΥΠΗΡΕΣΙΕΣ ΣΥΜΒΟΥΛΟΥ ΓΙΑ ΤΗΝ ΩΡΙΜΑΝΣΗ ΤΩΝ ΕΡΓΩΝ ΑΠΟΚΑΤΑΣΤΑΣΗΣ ΤΩΝ ΠΡΟΣΦΥΓΙΚΩΝ ΠΟΛΥΚΑΤΟΙΚΙΩΝ ΣΤΗ ΛΕΩΦΟΡΟ ΑΛΕΞΑΝΔΡΑΣ",7Θ3Ν7Λ7-Υ4Ν,11/10/2017 03:00:00,https://diavgeia.gov.gr/doc/7Θ3Ν7Λ7-Υ4Ν,ΠΡΑΞΗ,ΑΝΑΛΗΨΗ ΥΠΟΧΡΕΩΣΗΣ,7Θ3Ν7Λ7-Υ4Ν,"{\n ""relevance"": ""YES"",\n ""description"": ""Απόφαση δέσμευσης πίστωσης από την Περιφέρεια Αττικής για την πληρωμή υπηρεσιών συμβούλου για την ωρίμανση έργων αποκατάστασης των προσφυγικών πολυκατοικιών στη Λεωφόρο Αλεξάνδρας. Αναφέρεται κατανομή πίστωσης ανά έτος και σχετικοί κωδικοί προϋπολογισμού."",\n ""table"": [\n ""ΚΑΕ: 97690113701 (Ειδ. Φορέας 01.072) - «ΥΠΗΡΕΣΙΕΣ ΣΥΜΒΟΥΛΟΥ ΓΙΑ ΤΗΝ ΩΡΙΜΑΝΣΗ ΤΩΝ ΕΡΓΩΝ ΑΠΟΚΑΤΑΣΤΑΣΗΣ ΤΩΝ ΠΡΟΣΦΥΓΙΚΩΝ ΠΟΛΥΚΑΤΟΙΚΙΩΝ ΣΤΗ ΛΕΩΦΟΡΟ ΑΛΕΞΑΝΔΡΑΣ.»"",\n ""Συνολικό προϋπολογιζόμενο ποσό για υπηρεσίες συμβούλου: 1.000.000,00 € (αναφέρεται ως συνολικό ποσό)."",\n ""Κατανομή ανά έτος: 2017: 100.000,00 €"",\n ""Κατανομή ανά έτος: 2018: 160.000,00 €"",\n ""Κατανομή ανά έτος: 2019: 740.000,00 €"",\n ""Γραμμή υποχρεώσεων επόμενων ετών (01.072.976901137): 2018: 160.000,00 €; 2019: 740.000,00 €; Σύνολο μελλοντικών ετών: 900.000,00 €"",\n ""Υπόλοιπο προς ανάληψη που καταχωρήθηκε: 110.000,00 €""\n ],\n ""companies"": [],\n ""pages"": [\n 1\n ],\n ""extras"": ""Πρόκειται για διοικητική απόφαση ανάληψης υποχρέωσης/δέσμευσης πίστωσης (Απόφαση Οικονομικής Επιτροπής και σχετικές αποφάσεις υπηρεσιών). Περιλαμβάνονται αναφορές σε νομικά πλαίσια, εγκρίσεις προϋπολογισμού, αριθμούς πρωτοκόλλων και υπεύθυνους (Αντιπεριφερειάρχης κ.Χ. Καραμάνος). Δεν περιέχει πρακτικά συνεδριάσεων, αλλά τεκμηρίωση έγκρισης και κατανομής πιστώσεων για τις εργασίες ωρίμανσης/αποκατάστασης.""\n}"


csv cleaned rows: 24
saved: data_alexandras.csv
